# Metrics calculation

In [1]:
import pandas as pd
# summary_dataset = pd.read_csv("summary_dataset_with_bertscore.csv")
# columns = [column for column in summary_dataset.columns if "bertscore" not in column] 
# print(columns)
# summary_dataset = summary_dataset[columns]
# summary_dataset.to_csv("summary_dataset.csv", index=False)
summary_dataset = pd.read_csv("summary_dataset.csv")
summary_dataset.head()

,Unnamed: 0,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions
0,0,В Германии объяснили упоминание имени Путина н...,На этих выходных в Берлине прошли крупные акци...,Протестующие против антикоронавирусных мер нем...,В Берлине прошли массовые акции протеста проти...,В Берлине в субботу прошли массовые акции прот...,на этих выходных в берлине прошли крупные акци...
1,1,Делегации Израиля и США прибыли в ОАЭ для обсу...,Высокопоставленная американская и израильская ...,"Делегации Израиля и США прилетели в ОАЭ, где о...",Высокопоставленная американская и израильская ...,США и Израиль подписали историческое соглашени...,высокопоставленная американская и израильская ...
2,2,Оппозиция Белоруссии объявила о создании новой...,Одна из руководителей Координационного совета ...,Белорусская оппозиция в лице экс-кандидата в п...,Экс-кандидат в президенты Белоруссии Виктор Ба...,Координационный совет оппозиции Белоруссии объ...,одна из руководителей координационного совета ...
3,3,Россия считает крайне опасными действия США на...,Россия считает действия ВС США во время учений...,Действия американских ВС в Эстонии во время уч...,Россия считает действия ВС США во время учений...,"Российское посольство считает, что действия ам...",россия считает действия вс сша во время учений...
4,4,В России вступил в силу закон о внесудебном ба...,С 1 сентября в России вступают в силу поправки...,Поправки в российский закон «О банкротстве» вс...,С 1 сентября в России вступают в силу поправки...,С 1 сентября в России вступают в силу поправки...,с 1 сентября в россии вступают в силу поправки...


In [2]:
len(summary_dataset)

6793

## BERTScore

In [3]:
from evaluate import load
bertscore = load("bertscore")
files = [
    "mbart_predictions.txt",
    "mt5_predictions.txt",
    "summarunner_predictions.txt"
]

In [4]:
results = {}
references = list(summary_dataset["summary"])
for filename in files:
    predictions_field = filename.split(".")[0]
    predictions = list(summary_dataset[predictions_field])
    results[predictions_field] = bertscore.compute(predictions=predictions, references=references, lang="ru", verbose=True, model_type="microsoft/mdeberta-v3-base", nthreads=16, batch_size=16, use_fast_tokenizer=False)

calculating scores...
computing bert embedding.


  0%|          | 0/850 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/425 [00:00<?, ?it/s]

done in 361713.41 seconds, 0.02 sentences/sec
calculating scores...
computing bert embedding.


...

done in 361728.49 seconds, 0.02 sentences/sec
calculating scores...
computing bert embedding.


  0%|          | 0/850 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/425 [00:00<?, ?it/s]

done in 361749.75 seconds, 0.02 sentences/sec


In [5]:
data_keys = list(results.keys())
for data_key in data_keys:
    old_keys = list(results[data_key].keys())
    for old_metric_key in old_keys:
        results[data_key][f"{data_key}_bertscore_" + old_metric_key] = results[data_key].pop(old_metric_key)

In [6]:
print(results[filename.split(".")[0]].keys())

dict_keys(['summarunner_predictions_bertscore_precision', 'summarunner_predictions_bertscore_recall', 'summarunner_predictions_bertscore_f1', 'summarunner_predictions_bertscore_hashcode'])


In [7]:
print(len(results[filename.split(".")[0]][f"{filename.split('.')[0]}_bertscore_precision"]))

6793


In [8]:
new_data = {}
for data_key in results.keys():
    for metric_key in results[data_key].keys():
        if "hashcode" in metric_key:
            continue
        new_data[metric_key] = pd.Series(results[data_key][metric_key])
new_data_df = pd.DataFrame(new_data)
new_data_df.head()

,mbart_predictions_bertscore_precision,mbart_predictions_bertscore_recall,mbart_predictions_bertscore_f1,mt5_predictions_bertscore_precision,mt5_predictions_bertscore_recall,mt5_predictions_bertscore_f1,summarunner_predictions_bertscore_precision,summarunner_predictions_bertscore_recall,summarunner_predictions_bertscore_f1
0,0.711897,0.733298,0.722439,0.758551,0.656303,0.703732,0.665565,0.665563,0.665564
1,0.727948,0.682448,0.704464,0.744267,0.593580,0.660437,0.665305,0.651471,0.658316
2,0.690570,0.726228,0.707950,0.761381,0.561782,0.646527,0.667117,0.679452,0.673228
3,0.692291,0.760886,0.724970,0.772861,0.658800,0.711287,0.671986,0.732974,0.701156
4,0.714218,0.854085,0.777915,0.896744,0.812140,0.852348,0.676003,0.836950,0.747916


In [9]:
new_summary_dataset = pd.concat([summary_dataset, new_data_df.set_index(summary_dataset.index)], axis=1)
new_summary_dataset.head()

,Unnamed: 0,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,mbart_predictions_bertscore_precision,mbart_predictions_bertscore_recall,mbart_predictions_bertscore_f1,mt5_predictions_bertscore_precision,mt5_predictions_bertscore_recall,mt5_predictions_bertscore_f1,summarunner_predictions_bertscore_precision,summarunner_predictions_bertscore_recall,summarunner_predictions_bertscore_f1
0,0,В Германии объяснили упоминание имени Путина н...,На этих выходных в Берлине прошли крупные акци...,Протестующие против антикоронавирусных мер нем...,В Берлине прошли массовые акции протеста проти...,В Берлине в субботу прошли массовые акции прот...,на этих выходных в берлине прошли крупные акци...,0.711897,0.733298,0.722439,0.758551,0.656303,0.703732,0.665565,0.665563,0.665564
1,1,Делегации Израиля и США прибыли в ОАЭ для обсу...,Высокопоставленная американская и израильская ...,"Делегации Израиля и США прилетели в ОАЭ, где о...",Высокопоставленная американская и израильская ...,США и Израиль подписали историческое соглашени...,высокопоставленная американская и израильская ...,0.727948,0.682448,0.704464,0.744267,0.593580,0.660437,0.665305,0.651471,0.658316
2,2,Оппозиция Белоруссии объявила о создании новой...,Одна из руководителей Координационного совета ...,Белорусская оппозиция в лице экс-кандидата в п...,Экс-кандидат в президенты Белоруссии Виктор Ба...,Координационный совет оппозиции Белоруссии объ...,одна из руководителей координационного совета ...,0.690570,0.726228,0.707950,0.761381,0.561782,0.646527,0.667117,0.679452,0.673228
3,3,Россия считает крайне опасными действия США на...,Россия считает действия ВС США во время учений...,Действия американских ВС в Эстонии во время уч...,Россия считает действия ВС США во время учений...,"Российское посольство считает, что действия ам...",россия считает действия вс сша во время учений...,0.692291,0.760886,0.724970,0.772861,0.658800,0.711287,0.671986,0.732974,0.701156
4,4,В России вступил в силу закон о внесудебном ба...,С 1 сентября в России вступают в силу поправки...,Поправки в российский закон «О банкротстве» вс...,С 1 сентября в России вступают в силу поправки...,С 1 сентября в России вступают в силу поправки...,с 1 сентября в россии вступают в силу поправки...,0.714218,0.854085,0.777915,0.896744,0.812140,0.852348,0.676003,0.836950,0.747916


In [10]:
summary_dataset = pd.DataFrame(new_summary_dataset)
del new_summary_dataset

In [11]:
summary_dataset.head()

,Unnamed: 0,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,mbart_predictions_bertscore_precision,mbart_predictions_bertscore_recall,mbart_predictions_bertscore_f1,mt5_predictions_bertscore_precision,mt5_predictions_bertscore_recall,mt5_predictions_bertscore_f1,summarunner_predictions_bertscore_precision,summarunner_predictions_bertscore_recall,summarunner_predictions_bertscore_f1
0,0,В Германии объяснили упоминание имени Путина н...,На этих выходных в Берлине прошли крупные акци...,Протестующие против антикоронавирусных мер нем...,В Берлине прошли массовые акции протеста проти...,В Берлине в субботу прошли массовые акции прот...,на этих выходных в берлине прошли крупные акци...,0.711897,0.733298,0.722439,0.758551,0.656303,0.703732,0.665565,0.665563,0.665564
1,1,Делегации Израиля и США прибыли в ОАЭ для обсу...,Высокопоставленная американская и израильская ...,"Делегации Израиля и США прилетели в ОАЭ, где о...",Высокопоставленная американская и израильская ...,США и Израиль подписали историческое соглашени...,высокопоставленная американская и израильская ...,0.727948,0.682448,0.704464,0.744267,0.593580,0.660437,0.665305,0.651471,0.658316
2,2,Оппозиция Белоруссии объявила о создании новой...,Одна из руководителей Координационного совета ...,Белорусская оппозиция в лице экс-кандидата в п...,Экс-кандидат в президенты Белоруссии Виктор Ба...,Координационный совет оппозиции Белоруссии объ...,одна из руководителей координационного совета ...,0.690570,0.726228,0.707950,0.761381,0.561782,0.646527,0.667117,0.679452,0.673228
3,3,Россия считает крайне опасными действия США на...,Россия считает действия ВС США во время учений...,Действия американских ВС в Эстонии во время уч...,Россия считает действия ВС США во время учений...,"Российское посольство считает, что действия ам...",россия считает действия вс сша во время учений...,0.692291,0.760886,0.724970,0.772861,0.658800,0.711287,0.671986,0.732974,0.701156
4,4,В России вступил в силу закон о внесудебном ба...,С 1 сентября в России вступают в силу поправки...,Поправки в российский закон «О банкротстве» вс...,С 1 сентября в России вступают в силу поправки...,С 1 сентября в России вступают в силу поправки...,с 1 сентября в россии вступают в силу поправки...,0.714218,0.854085,0.777915,0.896744,0.812140,0.852348,0.676003,0.836950,0.747916


In [12]:
from statistics import mean 
from collections import defaultdict
score_stats = defaultdict(dict) if (not "score_stats" in locals() and not "score_stats" in globals()) else score_stats
for filename in files:
    predictions_field = filename.split(".")[0]
    
    mean_f1 = mean(results[predictions_field][f"{predictions_field}_bertscore_f1"])
    var_f1 = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_bertscore_f1"])) - mean_f1 ** 2
    
    mean_precision = mean(results[predictions_field][f"{predictions_field}_bertscore_precision"])
    var_precision = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_bertscore_precision"])) - mean_precision ** 2
    
    mean_recall = mean(results[predictions_field][f"{predictions_field}_bertscore_recall"])
    var_recall = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_bertscore_recall"])) - mean_recall ** 2
    
    score_stats[predictions_field].update({"mean_precision": mean_precision, "var_precision": var_precision, "mean_recall": mean_recall, "var_recall": var_recall, "mean_f1": mean_f1, "var_f1": var_f1})
print(dict(score_stats))

{'mbart_predictions': {'mean_precision': 0.692372342579568, 'var_precision': 0.002488098478101608, 'mean_recall': 0.6962039911762773, 'var_recall': 0.002787998923479862, 'mean_f1': 0.6936039534774038, 'var_f1': 0.0021470761969805685}, 'mt5_predictions': {'mean_precision': 0.7240417030525194, 'var_precision': 0.004430899025792079, 'mean_recall': 0.6168432836634025, 'var_recall': 0.002580475318054387, 'mean_f1': 0.6652786575710757, 'var_f1': 0.002742028795839624}, 'summarunner_predictions': {'mean_precision': 0.6593715531843919, 'var_precision': 0.0018870473652359276, 'mean_recall': 0.6711837874146075, 'var_recall': 0.002593393144960099, 'mean_f1': 0.6645703000557646, 'var_f1': 0.0017704626821319902}}


In [13]:
results.keys()

dict_keys(['mbart_predictions', 'mt5_predictions', 'summarunner_predictions'])

In [14]:
# stats_pd = pd.DataFrame(score_stats)

In [15]:
# stats_pdsummary_dataset_for_expert_evaluation.head(None)

In [16]:
summary_dataset.to_csv("summary_dataset_with_bertscore.csv", index=False)

## BLEU

In [17]:
import razdel
from tqdm import tqdm
from evaluate import load
# results = {}
bleu = load('bleu')
references = list(summary_dataset["summary"])

def a(x):
    tokens = list(razdel.tokenize(x))
    return [_.text for _ in tokens]
    # return x.split()

for filename in files:
    predictions_field = filename.split(".")[0]
    predictions = list(summary_dataset[predictions_field])
    # results[predictions_field] = rouge.compute(predictions=predictions, references=references, use_aggregator=False, tokenizer=lambda x : list(razdel.tokenize(x)))
    results[predictions_field]["bleu"] = []
    for ref, pred in tqdm(zip(references, predictions)):
        bl = bleu.compute(predictions=[pred], references=[ref], tokenizer=a)
        results[predictions_field]["bleu"].append(bl["bleu"])

6793it [00:11, 585.87it/s]
6793it [00:10, 629.18it/s]
6793it [00:11, 582.03it/s]


In [18]:
data_keys = list(results.keys())
print(data_keys)
for data_key in data_keys:
    old_keys = list(results[data_key].keys())
    print(old_keys)
    for old_metric_key in old_keys:
        results[data_key][f"{data_key}_" + old_metric_key] = results[data_key].pop(old_metric_key)

['mbart_predictions', 'mt5_predictions', 'summarunner_predictions']
['mbart_predictions_bertscore_precision', 'mbart_predictions_bertscore_recall', 'mbart_predictions_bertscore_f1', 'mbart_predictions_bertscore_hashcode', 'bleu']
['mt5_predictions_bertscore_precision', 'mt5_predictions_bertscore_recall', 'mt5_predictions_bertscore_f1', 'mt5_predictions_bertscore_hashcode', 'bleu']
['summarunner_predictions_bertscore_precision', 'summarunner_predictions_bertscore_recall', 'summarunner_predictions_bertscore_f1', 'summarunner_predictions_bertscore_hashcode', 'bleu']


In [19]:
print(results[filename.split(".")[0]].keys())

dict_keys(['summarunner_predictions_summarunner_predictions_bertscore_precision', 'summarunner_predictions_summarunner_predictions_bertscore_recall', 'summarunner_predictions_summarunner_predictions_bertscore_f1', 'summarunner_predictions_summarunner_predictions_bertscore_hashcode', 'summarunner_predictions_bleu'])


In [20]:
print(results[filename.split(".")[0]][f"{filename.split('.')[0]}_bleu"])

[0.06979764932727962, 0.0, 0.0, 0.09132199284647087, 0.12167273705115926, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09645191673650691, 0.0, 0.0, 0.0, 0.0, 0.12771461073274565, 0.09172084934569527, 0.1648638389103469, 0.10663135316463938, 0.0, 0.07810226520134339, 0.1030019432644136, 0.0, 0.06779174311659271, 0.0, 0.0, 0.06339306534122656, 0.0, 0.0, 0.28045224000158386, 0.0, 0.0, 0.18397764035505304, 0.0, 0.0, 0.0, 0.0, 0.04963338050445847, 0.0, 0.0, 0.0, 0.1062233760919317, 0.13585011613825984, 0.0, 0.08777581238861076, 0.0, 0.09601407002673158, 0.0, 0.061120520720478556, 0.0, 0.3091960906873235, 0.11708373934586765, 0.0, 0.0, 0.04253049257115781, 0.06733493544110147, 0.0, 0.05292788398968445, 0.0, 0.0483941549563098, 0.1671109463645597, 0.06798030034146445, 0.1810243813818477, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10118031664048839, 0.06471293257861663, 0.17174612744870413, 0.18329857013961992, 0.06842709524271848, 0.053840836210225784, 0.0, 0.0, 0.11345858732521114, 0.0, 0.0, 0.0, 0.16437176838

In [21]:
new_data = {}
for data_key in results.keys():
    for metric_key in results[data_key].keys():
        if not "bleu" in metric_key:
            continue
        new_data[metric_key] = pd.Series(results[data_key][metric_key])
new_data_df = pd.DataFrame(new_data)
new_data_df.head()

,mbart_predictions_bleu,mt5_predictions_bleu,summarunner_predictions_bleu
0,0.000000,0.000000,0.069798
1,0.000000,0.000000,0.000000
2,0.160395,0.000000,0.000000
3,0.130734,0.000000,0.091322
4,0.187633,0.323293,0.121673


In [22]:
new_summary_dataset = pd.concat([summary_dataset, new_data_df.set_index(summary_dataset.index)], axis=1)
new_summary_dataset.head()

,Unnamed: 0,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,mbart_predictions_bertscore_precision,mbart_predictions_bertscore_recall,mbart_predictions_bertscore_f1,mt5_predictions_bertscore_precision,mt5_predictions_bertscore_recall,mt5_predictions_bertscore_f1,summarunner_predictions_bertscore_precision,summarunner_predictions_bertscore_recall,summarunner_predictions_bertscore_f1,mbart_predictions_bleu,mt5_predictions_bleu,summarunner_predictions_bleu
0,0,В Германии объяснили упоминание имени Путина н...,На этих выходных в Берлине прошли крупные акци...,Протестующие против антикоронавирусных мер нем...,В Берлине прошли массовые акции протеста проти...,В Берлине в субботу прошли массовые акции прот...,на этих выходных в берлине прошли крупные акци...,0.711897,0.733298,0.722439,0.758551,0.656303,0.703732,0.665565,0.665563,0.665564,0.000000,0.000000,0.069798
1,1,Делегации Израиля и США прибыли в ОАЭ для обсу...,Высокопоставленная американская и израильская ...,"Делегации Израиля и США прилетели в ОАЭ, где о...",Высокопоставленная американская и израильская ...,США и Израиль подписали историческое соглашени...,высокопоставленная американская и израильская ...,0.727948,0.682448,0.704464,0.744267,0.593580,0.660437,0.665305,0.651471,0.658316,0.000000,0.000000,0.000000
2,2,Оппозиция Белоруссии объявила о создании новой...,Одна из руководителей Координационного совета ...,Белорусская оппозиция в лице экс-кандидата в п...,Экс-кандидат в президенты Белоруссии Виктор Ба...,Координационный совет оппозиции Белоруссии объ...,одна из руководителей координационного совета ...,0.690570,0.726228,0.707950,0.761381,0.561782,0.646527,0.667117,0.679452,0.673228,0.160395,0.000000,0.000000
3,3,Россия считает крайне опасными действия США на...,Россия считает действия ВС США во время учений...,Действия американских ВС в Эстонии во время уч...,Россия считает действия ВС США во время учений...,"Российское посольство считает, что действия ам...",россия считает действия вс сша во время учений...,0.692291,0.760886,0.724970,0.772861,0.658800,0.711287,0.671986,0.732974,0.701156,0.130734,0.000000,0.091322
4,4,В России вступил в силу закон о внесудебном ба...,С 1 сентября в России вступают в силу поправки...,Поправки в российский закон «О банкротстве» вс...,С 1 сентября в России вступают в силу поправки...,С 1 сентября в России вступают в силу поправки...,с 1 сентября в россии вступают в силу поправки...,0.714218,0.854085,0.777915,0.896744,0.812140,0.852348,0.676003,0.836950,0.747916,0.187633,0.323293,0.121673


In [23]:
summary_dataset = pd.DataFrame(new_summary_dataset)
del new_summary_dataset

In [24]:
summary_dataset.head()

,Unnamed: 0,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,mbart_predictions_bertscore_precision,mbart_predictions_bertscore_recall,mbart_predictions_bertscore_f1,mt5_predictions_bertscore_precision,mt5_predictions_bertscore_recall,mt5_predictions_bertscore_f1,summarunner_predictions_bertscore_precision,summarunner_predictions_bertscore_recall,summarunner_predictions_bertscore_f1,mbart_predictions_bleu,mt5_predictions_bleu,summarunner_predictions_bleu
0,0,В Германии объяснили упоминание имени Путина н...,На этих выходных в Берлине прошли крупные акци...,Протестующие против антикоронавирусных мер нем...,В Берлине прошли массовые акции протеста проти...,В Берлине в субботу прошли массовые акции прот...,на этих выходных в берлине прошли крупные акци...,0.711897,0.733298,0.722439,0.758551,0.656303,0.703732,0.665565,0.665563,0.665564,0.000000,0.000000,0.069798
1,1,Делегации Израиля и США прибыли в ОАЭ для обсу...,Высокопоставленная американская и израильская ...,"Делегации Израиля и США прилетели в ОАЭ, где о...",Высокопоставленная американская и израильская ...,США и Израиль подписали историческое соглашени...,высокопоставленная американская и израильская ...,0.727948,0.682448,0.704464,0.744267,0.593580,0.660437,0.665305,0.651471,0.658316,0.000000,0.000000,0.000000
2,2,Оппозиция Белоруссии объявила о создании новой...,Одна из руководителей Координационного совета ...,Белорусская оппозиция в лице экс-кандидата в п...,Экс-кандидат в президенты Белоруссии Виктор Ба...,Координационный совет оппозиции Белоруссии объ...,одна из руководителей координационного совета ...,0.690570,0.726228,0.707950,0.761381,0.561782,0.646527,0.667117,0.679452,0.673228,0.160395,0.000000,0.000000
3,3,Россия считает крайне опасными действия США на...,Россия считает действия ВС США во время учений...,Действия американских ВС в Эстонии во время уч...,Россия считает действия ВС США во время учений...,"Российское посольство считает, что действия ам...",россия считает действия вс сша во время учений...,0.692291,0.760886,0.724970,0.772861,0.658800,0.711287,0.671986,0.732974,0.701156,0.130734,0.000000,0.091322
4,4,В России вступил в силу закон о внесудебном ба...,С 1 сентября в России вступают в силу поправки...,Поправки в российский закон «О банкротстве» вс...,С 1 сентября в России вступают в силу поправки...,С 1 сентября в России вступают в силу поправки...,с 1 сентября в россии вступают в силу поправки...,0.714218,0.854085,0.777915,0.896744,0.812140,0.852348,0.676003,0.836950,0.747916,0.187633,0.323293,0.121673


In [25]:
# from statistics import mean 
# from collections import defaultdict
# score_stats = defaultdict(dict) if (not "score_stats" in locals() and not "score_stats" in globals()) else score_stats
# for filename in files:
#     predictions_field = filename.split(".")[0]
#     mean_rouge1 = mean(results[predictions_field][f"{predictions_field}_rouge1"])
#     var_rouge1 = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rouge1"])) - mean_rouge1 ** 2
    
#     mean_rouge2 = mean(results[predictions_field][f"{predictions_field}_rouge2"])
#     var_rouge2 = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rouge2"])) - mean_rouge2 ** 2
    
#     mean_rougeL = mean(results[predictions_field][f"{predictions_field}_rougeL"])
#     var_rougeL = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rougeL"])) - mean_rougeL ** 2
    
#     mean_rougeLsum = mean(results[predictions_field][f"{predictions_field}_rougeLsum"])
#     var_rougeLsum = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rougeLsum"])) - mean_rougeLsum ** 2
#     score_stats[predictions_field].update({"mean_rouge1": mean_rouge1, "var_rouge1": var_rouge1, "mean_rouge2": mean_rouge2, "var_rouge2": var_rouge2, "mean_rougeL": mean_rougeL, "var_rougeL": var_rougeL, "mean_rougeLsum": mean_rougeLsum, "var_rougeLsum": var_rougeLsum})
# print(dict(score_stats))

In [26]:
# stats_pd = pd.DataFrame(score_stats)

In [27]:
# stats_pd.head(len(stats_pd))

In [28]:
summary_dataset.to_csv("summary_dataset_with_bertscore_bleu.csv", index=False)

## ROUGE

In [29]:
import pandas as pd
summary_dataset = pd.read_csv("summary_dataset_with_bertscore_bleu.csv")
files = [
    "mbart_predictions.txt",
    "mt5_predictions.txt",
    "summarunner_predictions.txt"
]

In [30]:
import razdel
from evaluate import load
# results = {}
rouge = load('rouge')
references = list(summary_dataset["summary"])

def a(x):
    tokens = list(razdel.tokenize(x))
    return [_.text for _ in tokens]
    # return x.split()

for filename in files:
    predictions_field = filename.split(".")[0]
    predictions = list(summary_dataset[predictions_field])
    # results[predictions_field] = rouge.compute(predictions=predictions, references=references, use_aggregator=False, tokenizer=lambda x : list(razdel.tokenize(x)))
    results[predictions_field] = rouge.compute(predictions=predictions, references=references, use_aggregator=False, tokenizer=a)

In [31]:
data_keys = list(results.keys())
for data_key in data_keys:
    old_keys = list(results[data_key].keys())
    for old_metric_key in old_keys:
        results[data_key][f"{data_key}_" + old_metric_key] = results[data_key].pop(old_metric_key)

In [32]:
print(results[filename.split(".")[0]].keys())

dict_keys(['summarunner_predictions_rouge1', 'summarunner_predictions_rouge2', 'summarunner_predictions_rougeL', 'summarunner_predictions_rougeLsum'])


In [52]:
print(len(results[filename.split(".")[0]][f"{filename.split('.')[0]}_rouge1"]))

6793


In [53]:
new_data = {}
for data_key in results.keys():
    for metric_key in results[data_key].keys():
        if "hashcode" in metric_key:
            continue
        new_data[metric_key] = pd.Series(results[data_key][metric_key])
new_data_df = pd.DataFrame(new_data)
new_data_df.head()

,mbart_predictions_rouge1,mbart_predictions_rouge2,mbart_predictions_rougeL,mbart_predictions_rougeLsum,mt5_predictions_rouge1,mt5_predictions_rouge2,mt5_predictions_rougeL,mt5_predictions_rougeLsum,summarunner_predictions_rouge1,summarunner_predictions_rouge2,summarunner_predictions_rougeL,summarunner_predictions_rougeLsum
0,0.292683,0.099174,0.178862,0.178862,0.317073,0.050000,0.146341,0.146341,0.272727,0.092593,0.181818,0.181818
1,0.258621,0.052632,0.189655,0.189655,0.164706,0.048193,0.164706,0.164706,0.222222,0.034783,0.153846,0.153846
2,0.358974,0.191304,0.324786,0.324786,0.135593,0.035088,0.135593,0.135593,0.306306,0.110092,0.252252,0.252252
3,0.377953,0.176000,0.299213,0.299213,0.264706,0.090909,0.176471,0.176471,0.346457,0.128000,0.267717,0.267717
4,0.436975,0.290598,0.352941,0.352941,0.602740,0.422535,0.465753,0.465753,0.391304,0.220588,0.289855,0.289855


In [33]:
new_summary_dataset = pd.concat([summary_dataset, new_data_df.set_index(new_data_df.index)], axis=1)
new_summary_dataset.head()

,Unnamed: 0,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,mbart_predictions_bertscore_precision,mbart_predictions_bertscore_recall,mbart_predictions_bertscore_f1,...,mt5_predictions_bertscore_f1,summarunner_predictions_bertscore_precision,summarunner_predictions_bertscore_recall,summarunner_predictions_bertscore_f1,mbart_predictions_bleu,mt5_predictions_bleu,summarunner_predictions_bleu,mbart_predictions_bleu,mt5_predictions_bleu,summarunner_predictions_bleu
0,0,В Германии объяснили упоминание имени Путина н...,На этих выходных в Берлине прошли крупные акци...,Протестующие против антикоронавирусных мер нем...,В Берлине прошли массовые акции протеста проти...,В Берлине в субботу прошли массовые акции прот...,на этих выходных в берлине прошли крупные акци...,0.711897,0.733298,0.722439,...,0.703732,0.665565,0.665563,0.665564,0.000000,0.000000,0.069798,0.000000,0.000000,0.069798
1,1,Делегации Израиля и США прибыли в ОАЭ для обсу...,Высокопоставленная американская и израильская ...,"Делегации Израиля и США прилетели в ОАЭ, где о...",Высокопоставленная американская и израильская ...,США и Израиль подписали историческое соглашени...,высокопоставленная американская и израильская ...,0.727948,0.682448,0.704464,...,0.660437,0.665305,0.651471,0.658316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,Оппозиция Белоруссии объявила о создании новой...,Одна из руководителей Координационного совета ...,Белорусская оппозиция в лице экс-кандидата в п...,Экс-кандидат в президенты Белоруссии Виктор Ба...,Координационный совет оппозиции Белоруссии объ...,одна из руководителей координационного совета ...,0.690570,0.726228,0.707950,...,0.646527,0.667117,0.679452,0.673228,0.160395,0.000000,0.000000,0.160395,0.000000,0.000000
3,3,Россия считает крайне опасными действия США на...,Россия считает действия ВС США во время учений...,Действия американских ВС в Эстонии во время уч...,Россия считает действия ВС США во время учений...,"Российское посольство считает, что действия ам...",россия считает действия вс сша во время учений...,0.692291,0.760886,0.724970,...,0.711287,0.671986,0.732974,0.701156,0.130734,0.000000,0.091322,0.130734,0.000000,0.091322
4,4,В России вступил в силу закон о внесудебном ба...,С 1 сентября в России вступают в силу поправки...,Поправки в российский закон «О банкротстве» вс...,С 1 сентября в России вступают в силу поправки...,С 1 сентября в России вступают в силу поправки...,с 1 сентября в россии вступают в силу поправки...,0.714218,0.854085,0.777915,...,0.852348,0.676003,0.836950,0.747916,0.187633,0.323293,0.121673,0.187633,0.323293,0.121673


In [34]:
summary_dataset = pd.DataFrame(new_summary_dataset)
del new_summary_dataset

In [35]:
summary_dataset.head()

,Unnamed: 0,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,mbart_predictions_bertscore_precision,mbart_predictions_bertscore_recall,mbart_predictions_bertscore_f1,...,mt5_predictions_bertscore_f1,summarunner_predictions_bertscore_precision,summarunner_predictions_bertscore_recall,summarunner_predictions_bertscore_f1,mbart_predictions_bleu,mt5_predictions_bleu,summarunner_predictions_bleu,mbart_predictions_bleu,mt5_predictions_bleu,summarunner_predictions_bleu
0,0,В Германии объяснили упоминание имени Путина н...,На этих выходных в Берлине прошли крупные акци...,Протестующие против антикоронавирусных мер нем...,В Берлине прошли массовые акции протеста проти...,В Берлине в субботу прошли массовые акции прот...,на этих выходных в берлине прошли крупные акци...,0.711897,0.733298,0.722439,...,0.703732,0.665565,0.665563,0.665564,0.000000,0.000000,0.069798,0.000000,0.000000,0.069798
1,1,Делегации Израиля и США прибыли в ОАЭ для обсу...,Высокопоставленная американская и израильская ...,"Делегации Израиля и США прилетели в ОАЭ, где о...",Высокопоставленная американская и израильская ...,США и Израиль подписали историческое соглашени...,высокопоставленная американская и израильская ...,0.727948,0.682448,0.704464,...,0.660437,0.665305,0.651471,0.658316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,Оппозиция Белоруссии объявила о создании новой...,Одна из руководителей Координационного совета ...,Белорусская оппозиция в лице экс-кандидата в п...,Экс-кандидат в президенты Белоруссии Виктор Ба...,Координационный совет оппозиции Белоруссии объ...,одна из руководителей координационного совета ...,0.690570,0.726228,0.707950,...,0.646527,0.667117,0.679452,0.673228,0.160395,0.000000,0.000000,0.160395,0.000000,0.000000
3,3,Россия считает крайне опасными действия США на...,Россия считает действия ВС США во время учений...,Действия американских ВС в Эстонии во время уч...,Россия считает действия ВС США во время учений...,"Российское посольство считает, что действия ам...",россия считает действия вс сша во время учений...,0.692291,0.760886,0.724970,...,0.711287,0.671986,0.732974,0.701156,0.130734,0.000000,0.091322,0.130734,0.000000,0.091322
4,4,В России вступил в силу закон о внесудебном ба...,С 1 сентября в России вступают в силу поправки...,Поправки в российский закон «О банкротстве» вс...,С 1 сентября в России вступают в силу поправки...,С 1 сентября в России вступают в силу поправки...,с 1 сентября в россии вступают в силу поправки...,0.714218,0.854085,0.777915,...,0.852348,0.676003,0.836950,0.747916,0.187633,0.323293,0.121673,0.187633,0.323293,0.121673


Посчитаем статистики для представления о качестве датасета

In [36]:
from statistics import mean 
from collections import defaultdict
score_stats = defaultdict(dict) if (not "score_stats" in locals() and not "score_stats" in globals()) else score_stats
for filename in files:
    predictions_field = filename.split(".")[0]
    mean_rouge1 = mean(results[predictions_field][f"{predictions_field}_rouge1"])
    var_rouge1 = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rouge1"])) - mean_rouge1 ** 2
    
    mean_rouge2 = mean(results[predictions_field][f"{predictions_field}_rouge2"])
    var_rouge2 = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rouge2"])) - mean_rouge2 ** 2
    
    mean_rougeL = mean(results[predictions_field][f"{predictions_field}_rougeL"])
    var_rougeL = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rougeL"])) - mean_rougeL ** 2
    
    mean_rougeLsum = mean(results[predictions_field][f"{predictions_field}_rougeLsum"])
    var_rougeLsum = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rougeLsum"])) - mean_rougeLsum ** 2
    score_stats[predictions_field].update({"mean_rouge1": mean_rouge1, "var_rouge1": var_rouge1, "mean_rouge2": mean_rouge2, "var_rouge2": var_rouge2, "mean_rougeL": mean_rougeL, "var_rougeL": var_rougeL, "mean_rougeLsum": mean_rougeLsum, "var_rougeLsum": var_rougeLsum})
print(dict(score_stats))

{'mbart_predictions': {'mean_precision': 0.692372342579568, 'var_precision': 0.002488098478101608, 'mean_recall': 0.6962039911762773, 'var_recall': 0.002787998923479862, 'mean_f1': 0.6936039534774038, 'var_f1': 0.0021470761969805685, 'mean_rouge1': 0.31801428929276543, 'var_rouge1': 0.009495205444211535, 'mean_rouge2': 0.11031830313712905, 'var_rouge2': 0.008153947814415904, 'mean_rougeL': 0.22786116051309793, 'var_rougeL': 0.007198383393708874, 'mean_rougeLsum': 0.22786116051309793, 'var_rougeLsum': 0.007198383393708874}, 'mt5_predictions': {'mean_precision': 0.7240417030525194, 'var_precision': 0.004430899025792079, 'mean_recall': 0.6168432836634025, 'var_recall': 0.002580475318054387, 'mean_f1': 0.6652786575710757, 'var_f1': 0.002742028795839624, 'mean_rouge1': 0.21508826169541578, 'var_rouge1': 0.009881811467355672, 'mean_rouge2': 0.07156625942695444, 'var_rouge2': 0.005848079935605663, 'mean_rougeL': 0.17260913142505446, 'var_rougeL': 0.006988653580895881, 'mean_rougeLsum': 0.1726

In [37]:
stats_pd = pd.DataFrame(score_stats)

In [38]:
stats_pd.head(len(stats_pd))

,mbart_predictions,mt5_predictions,summarunner_predictions
mean_precision,0.692372,0.724042,0.659372
var_precision,0.002488,0.004431,0.001887
mean_recall,0.696204,0.616843,0.671184
var_recall,0.002788,0.002580,0.002593
mean_f1,0.693604,0.665279,0.664570
var_f1,0.002147,0.002742,0.001770
mean_rouge1,0.318014,0.215088,0.266036
var_rouge1,0.009495,0.009882,0.006447
mean_rouge2,0.110318,0.071566,0.065286
var_rouge2,0.008154,0.005848,0.004327


In [39]:
summary_dataset.to_csv("summary_dataset_with_bertscore_bleu_rouge.csv", index=False)

## METEOR

https://huggingface.co/spaces/evaluate-metric/meteor

In [40]:
import pandas as pd
summary_dataset = pd.read_csv("summary_dataset_with_bertscore_bleu_rouge.csv")
files = [
    "mbart_predictions.txt",
    "mt5_predictions.txt",
    "summarunner_predictions.txt",
]

In [41]:
from tqdm import tqdm
from threading import Thread
from evaluate import load
import os, contextlib


def calculate_meteor(result_list, predictions, references, thread_no, step):
    meteor = None
    # with open(os.devnull, 'w') as devnull:
    #     with contextlib.redirect_stdout(devnull), contextlib.redirect_stderr(devnull):
    meteor = load('meteor')
    index = thread_no
    disable_tqdm = (thread_no != 0)
    
    with tqdm(total=len(predictions), disable=disable_tqdm) as pbar:
        pbar.update(index)
        while index < len(predictions):
            result_list[index] = meteor.compute(predictions=[predictions[index]], references=[references[index]])
            index += step
            pbar.update(step)
    print(f"Thread {thread_no} finished")

results = {}
references = list(summary_dataset["summary"])
for filename in files:
    predictions_field = filename.split(".")[0]
    predictions = list(summary_dataset[predictions_field])
    results[predictions_field] = [0] * len(predictions)
    
    thread_num = 32
    thread_pool = []
    for thread in range(thread_num):
        thread_pool.append(Thread(target=calculate_meteor, args=[results[predictions_field], predictions, references, thread, thread_num], daemon=True))
        thread_pool[-1].start()

    for thread in range(thread_num):
        thread_pool[thread].join()

[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data] Downloading package wordnet to /h

Thread 0 finished
Thread 31 finished
Thread 16 finished
Thread 30 finished
Thread 26 finished
Thread 20 finished
Thread 15 finished
Thread 24 finished
Thread 23 finished
Thread 13 finished
Thread 9 finished
Thread 10 finished
Thread 14 finished
Thread 22 finished
Thread 4 finished
Thread 21 finished
Thread 29 finished
Thread 6 finished
Thread 7 finished
Thread 18 finished
Thread 19 finished
Thread 17 finished
Thread 1 finished
Thread 11 finished
Thread 12 finished
Thread 3 finished
Thread 2 finished
Thread 28 finished
Thread 27 finished
Thread 25 finished
Thread 8 finished
Thread 5 finished


[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/skatori/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/skatori/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading pack

Thread 19 finished


 77%|████████████████████████████████████████▉            | 5248/6793 [00:10<00:01, 979.08it/s]

...

 97%|███████████████████████████████████████████████████▏ | 6560/6793 [00:14<00:00, 525.40it/s]

Thread 13 finished
Thread 17 finished


6816it [00:14, 467.39it/s]                                                                     

Thread 7 finished
Thread 0 finished
Thread 26 finished


Thread 22 finished
Thread 23 finished
Thread 27 finished
Thread 8 finished
Thread 14 finished
Thread 19 finished
Thread 2 finished
Thread 1 finished
Thread 31 finished
Thread 30 finished
Thread 28 finished
Thread 25 finished
Thread 20 finished
Thread 18 finished
Thread 16 finished
Thread 15 finished
Thread 4 finished
Thread 11 finished
Thread 9 finished


In [42]:
for key in results.keys():
    results[key] = {f"{key}_meteor": [val["meteor"] for val in results[key]]}
    print(len(results[key]))

1
1
1


In [43]:
new_data = {}
for data_key in results.keys():
    for metric_key in results[data_key].keys():
        if "hashcode" in metric_key:
            continue
        new_data[metric_key] = pd.Series(results[data_key][metric_key])
new_data_df = pd.DataFrame(new_data)
new_data_df.head()

,mbart_predictions_meteor,mt5_predictions_meteor,summarunner_predictions_meteor
0,0.327036,0.209577,0.298614
1,0.155565,0.087664,0.140027
2,0.332507,0.069983,0.336473
3,0.399261,0.186599,0.404262
4,0.601744,0.576856,0.570060


In [44]:
new_summary_dataset = pd.concat([summary_dataset, new_data_df.set_index(summary_dataset.index)], axis=1)
new_summary_dataset.head()

,Unnamed: 0,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,mbart_predictions_bertscore_precision,mbart_predictions_bertscore_recall,mbart_predictions_bertscore_f1,...,summarunner_predictions_bertscore_f1,mbart_predictions_bleu,mt5_predictions_bleu,summarunner_predictions_bleu,mbart_predictions_bleu.1,mt5_predictions_bleu.1,summarunner_predictions_bleu.1,mbart_predictions_meteor,mt5_predictions_meteor,summarunner_predictions_meteor
0,0,В Германии объяснили упоминание имени Путина н...,На этих выходных в Берлине прошли крупные акци...,Протестующие против антикоронавирусных мер нем...,В Берлине прошли массовые акции протеста проти...,В Берлине в субботу прошли массовые акции прот...,на этих выходных в берлине прошли крупные акци...,0.711897,0.733298,0.722439,...,0.665564,0.000000,0.000000,0.069798,0.000000,0.000000,0.069798,0.327036,0.209577,0.298614
1,1,Делегации Израиля и США прибыли в ОАЭ для обсу...,Высокопоставленная американская и израильская ...,"Делегации Израиля и США прилетели в ОАЭ, где о...",Высокопоставленная американская и израильская ...,США и Израиль подписали историческое соглашени...,высокопоставленная американская и израильская ...,0.727948,0.682448,0.704464,...,0.658316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.155565,0.087664,0.140027
2,2,Оппозиция Белоруссии объявила о создании новой...,Одна из руководителей Координационного совета ...,Белорусская оппозиция в лице экс-кандидата в п...,Экс-кандидат в президенты Белоруссии Виктор Ба...,Координационный совет оппозиции Белоруссии объ...,одна из руководителей координационного совета ...,0.690570,0.726228,0.707950,...,0.673228,0.160395,0.000000,0.000000,0.160395,0.000000,0.000000,0.332507,0.069983,0.336473
3,3,Россия считает крайне опасными действия США на...,Россия считает действия ВС США во время учений...,Действия американских ВС в Эстонии во время уч...,Россия считает действия ВС США во время учений...,"Российское посольство считает, что действия ам...",россия считает действия вс сша во время учений...,0.692291,0.760886,0.724970,...,0.701156,0.130734,0.000000,0.091322,0.130734,0.000000,0.091322,0.399261,0.186599,0.404262
4,4,В России вступил в силу закон о внесудебном ба...,С 1 сентября в России вступают в силу поправки...,Поправки в российский закон «О банкротстве» вс...,С 1 сентября в России вступают в силу поправки...,С 1 сентября в России вступают в силу поправки...,с 1 сентября в россии вступают в силу поправки...,0.714218,0.854085,0.777915,...,0.747916,0.187633,0.323293,0.121673,0.187633,0.323293,0.121673,0.601744,0.576856,0.570060


In [45]:
summary_dataset = pd.DataFrame(new_summary_dataset)
del new_summary_dataset

In [46]:
summary_dataset.head()

,Unnamed: 0,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,mbart_predictions_bertscore_precision,mbart_predictions_bertscore_recall,mbart_predictions_bertscore_f1,...,summarunner_predictions_bertscore_f1,mbart_predictions_bleu,mt5_predictions_bleu,summarunner_predictions_bleu,mbart_predictions_bleu.1,mt5_predictions_bleu.1,summarunner_predictions_bleu.1,mbart_predictions_meteor,mt5_predictions_meteor,summarunner_predictions_meteor
0,0,В Германии объяснили упоминание имени Путина н...,На этих выходных в Берлине прошли крупные акци...,Протестующие против антикоронавирусных мер нем...,В Берлине прошли массовые акции протеста проти...,В Берлине в субботу прошли массовые акции прот...,на этих выходных в берлине прошли крупные акци...,0.711897,0.733298,0.722439,...,0.665564,0.000000,0.000000,0.069798,0.000000,0.000000,0.069798,0.327036,0.209577,0.298614
1,1,Делегации Израиля и США прибыли в ОАЭ для обсу...,Высокопоставленная американская и израильская ...,"Делегации Израиля и США прилетели в ОАЭ, где о...",Высокопоставленная американская и израильская ...,США и Израиль подписали историческое соглашени...,высокопоставленная американская и израильская ...,0.727948,0.682448,0.704464,...,0.658316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.155565,0.087664,0.140027
2,2,Оппозиция Белоруссии объявила о создании новой...,Одна из руководителей Координационного совета ...,Белорусская оппозиция в лице экс-кандидата в п...,Экс-кандидат в президенты Белоруссии Виктор Ба...,Координационный совет оппозиции Белоруссии объ...,одна из руководителей координационного совета ...,0.690570,0.726228,0.707950,...,0.673228,0.160395,0.000000,0.000000,0.160395,0.000000,0.000000,0.332507,0.069983,0.336473
3,3,Россия считает крайне опасными действия США на...,Россия считает действия ВС США во время учений...,Действия американских ВС в Эстонии во время уч...,Россия считает действия ВС США во время учений...,"Российское посольство считает, что действия ам...",россия считает действия вс сша во время учений...,0.692291,0.760886,0.724970,...,0.701156,0.130734,0.000000,0.091322,0.130734,0.000000,0.091322,0.399261,0.186599,0.404262
4,4,В России вступил в силу закон о внесудебном ба...,С 1 сентября в России вступают в силу поправки...,Поправки в российский закон «О банкротстве» вс...,С 1 сентября в России вступают в силу поправки...,С 1 сентября в России вступают в силу поправки...,с 1 сентября в россии вступают в силу поправки...,0.714218,0.854085,0.777915,...,0.747916,0.187633,0.323293,0.121673,0.187633,0.323293,0.121673,0.601744,0.576856,0.570060


Посчитаем статистики для представления о качестве датасета

In [47]:
from collections import defaultdict

score_stats = defaultdict(dict) if (not "score_stats" in locals() and not "score_stats" in globals()) else score_stats
for filename in files:
    predictions_field = filename.split(".")[0]
    score_stats[predictions_field].update({f"meteor": sum(results[predictions_field][f"{predictions_field}_meteor"]) / len(results[predictions_field][f"{predictions_field}_meteor"])})

In [48]:
stats_pd = pd.DataFrame(score_stats)

In [49]:
stats_pd.head(len(stats_pd))

,mbart_predictions,mt5_predictions,summarunner_predictions
mean_precision,0.692372,0.724042,0.659372
var_precision,0.002488,0.004431,0.001887
mean_recall,0.696204,0.616843,0.671184
var_recall,0.002788,0.002580,0.002593
mean_f1,0.693604,0.665279,0.664570
var_f1,0.002147,0.002742,0.001770
mean_rouge1,0.318014,0.215088,0.266036
var_rouge1,0.009495,0.009882,0.006447
mean_rouge2,0.110318,0.071566,0.065286
var_rouge2,0.008154,0.005848,0.004327


In [50]:
summary_dataset.to_csv("summary_dataset_with_bertscore_bleu_rouge_meteor.csv", index=False)